In [4]:
import pandas as pd
import os

# Assign paths
path_protime = r'./Protime'
path_agency = r'./Agency'

# List files in directories
folder_protime = os.listdir(path_protime)
folder_agency = os.listdir(path_agency)

def concat_data(path, folder, file_extension=None, starts_with=None):
    # Filter files based on extension and starting name
    filtered_files = [file for file in folder if (file_extension is None or file.endswith(file_extension))]# and (starts_with is None or file.startswith(starts_with))]
    
    # Concatenate files or read single file
    if len(filtered_files) > 1:
        df = pd.concat([pd.read_excel(os.path.join(path, file)) for file in filtered_files])
    elif len(filtered_files) == 1:
        df = pd.read_excel(os.path.join(path, filtered_files[0]))
    else:
        df = pd.DataFrame()  # Return an empty DataFrame if no files match
    return df

# Load data
agency = concat_data(path_agency, folder_agency, '.xlsx')
protime = concat_data(path_protime, folder_protime, '.xlsx')

# Ensure correct date format and calculate week number
agency['Datum'] = pd.to_datetime(agency['Datum'])
agency['Week_Number'] = agency['Datum'].dt.strftime('%Y-W%U')

protime['Date'] = pd.to_datetime(protime['Date'])
protime['Week_Number'] = protime['Date'].dt.strftime('%Y-W%U')

# Rename columns for clarity
agency = agency.rename(columns={'Uren': 'Agency hours'})
protime = protime.rename(columns={'Hours (Dec)': 'GXO Hours'})

# Aggregate data by date and employee name
agency_aggregated = agency.groupby(['Naam Medewerker','Datum'])['Agency hours'].sum().reset_index()
protime_aggregated = protime.groupby(['Full Name', 'Date'])['GXO Hours'].sum().reset_index()

# print(agency_aggregated.head())
# print(protime_aggregated.head())

# Merging based on dates and employee names
merged_df = pd.merge(agency, protime, 
                     left_on=['Datum', 'Naam Medewerker'], 
                     right_on=['Date', 'Full Name'], 
                     how='outer', indicator=True)

# Fill NaN values with 0 to handle missing data from the outer merge
merged_df['Agency hours'].fillna(0, inplace=True)
merged_df['GXO Hours'].fillna(0, inplace=True)

# Convert hours to minutes
merged_df['Agency hours'] = merged_df['Agency hours'] * 60
merged_df['GXO Hours'] = merged_df['GXO Hours'] * 60

# Create a column to check if the minutes match
merged_df['Hours_Difference'] = merged_df['Agency hours'] - merged_df['GXO Hours']

# Add a column to check the source of the difference
merged_df['Source'] = merged_df.apply(
    lambda row: 'Agency Only' if row['_merge'] == 'left_only' else (
                'Protime Only' if row['_merge'] == 'right_only' else (
                'Mismatch' if row['Hours_Difference'] != 0 else 'Match')),
    axis=1
)

# Filter rows where there is a mismatch or a missing value in one of the datasets
diff_df = merged_df[merged_df['Source'] != 'Match']

# Exclude rows where the difference is less than 2 minutes
diff_df = diff_df[diff_df['Hours_Difference'].abs() >= 2]

# Rename columns with units for clarity
diff_df.rename(columns={
    'Datum': 'Date',
    'Agency hours': 'Agency hours (minutes)',
    'GXO Hours': 'GXO Hours (minutes)',
    'Hours_Difference': 'Hours Difference (minutes)'
}, inplace=True)

# Display the rows with discrepancies and the source
print(diff_df)

# Save the results to an Excel file
diff_df.to_excel('Long_Report.xlsx', index=False)


     Invoice nr Kostenplaats site   Naam Medewerker  Personeels nummer  \
0           NaN               NaN               NaN                NaN   
3    95290085.0         GXO Venlo  Adrian Ciobanita           388412.0   
4    95290085.0         GXO Venlo  Adrian Ciobanita           388412.0   
6           NaN               NaN               NaN                NaN   
7           NaN               NaN               NaN                NaN   
..          ...               ...               ...                ...   
747         NaN               NaN               NaN                NaN   
752         NaN               NaN               NaN                NaN   
753         NaN               NaN               NaN                NaN   
754  95290085.0         GXO Venlo      Zyta Dlugosz           339973.0   
757  95290085.0         GXO Venlo      Zyta Dlugosz           339973.0   

      Toeslag       Date  Gewerkte week  Agency hours (minutes)  \
0         NaN        NaT            NaN     

c:\Users\boyan.todorov\Desktop\PDFCOMPARE\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\boyan.todorov\AppData\Local\Temp\ipykernel_16068\1217478499.py:54: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['Agency hours'].fillna(0, inplace=True)
C:\Users\boyan.todorov\AppData\Local\Temp\ipykernel_16068\1217478499.py:55: FutureWarning: A value is trying to be 

In [9]:

agency_aggregated = agency.groupby(['Datum'])['Uren'].sum().reset_index()
protime_aggregated = protime.groupby(['Date'])['Hours (Dec)'].sum().reset_index()

print(agency_aggregated.head())
print(protime_aggregated.head())

# Merging based on dates and employee names
merged_df = pd.merge(agency_aggregated, protime_aggregated, 
                     left_on=['Datum', 'Naam Medewerker'], 
                     right_on=['Date', 'Full Name'], 
                     how='outer', indicator=True)

# Fill NaN values with 0 to handle missing data from the outer merge
merged_df['Uren'].fillna(0, inplace=True)
merged_df['Hours (Dec)'].fillna(0, inplace=True)

# Create a column to check if the hours match
merged_df['Hours_Difference'] = merged_df['Uren'] - merged_df['Hours (Dec)']

# Add a column to check the source of the difference
merged_df['Source'] = merged_df.apply(
    lambda row: 'Agency Only' if row['_merge'] == 'left_only' else (
                'Protime Only' if row['_merge'] == 'right_only' else (
                'Mismatch' if row['Hours_Difference'] != 0 else 'Match')),
    axis=1
)

# Filter rows where there is a mismatch or a missing value in one of the datasets
diff_df = merged_df[merged_df['Source'] != 'Match']

# Drop unnecessary columns for cleaner output
diff_df = diff_df.drop(columns=['_merge', 'Date', 'Full Name'])

# Display the rows with discrepancies and the source
print(diff_df)

diff_df.to_excel('Long_Report.xlsx')

KeyError: 'Column not found: Uren'

In [6]:

agency_aggregated = agency.groupby(['Datum', 'Naam Medewerker'])['Uren'].sum().reset_index()
protime_aggregated = protime.groupby(['Date', 'Full Name'])['Hours (Dec)'].sum().reset_index()

# Merging the two DataFrames based on dates and employee names (inner join to match both)
merged_df = pd.merge(agency_aggregated, protime_aggregated, 
                     left_on=['Datum', 'Naam Medewerker'], 
                     right_on=['Date', 'Full Name'], 
                     how='inner')  # Only keep rows that exist in both datasets

# Create a column to check the difference in hours
merged_df['Hours_Difference'] = merged_df['Uren'] - merged_df['Hours (Dec)']

# Filter rows where the difference is greater than 2 minutes (0.0333 hours)
diff_df = merged_df[abs(merged_df['Hours_Difference']) > 0.0333]

# Format the 'Datum' column to remove the time and display only the date
diff_df['Datum'] = pd.to_datetime(diff_df['Datum']).dt.strftime('%Y-%m-%d')

# Round the decimal values for hours to 2 decimal places for better readability
diff_df['Hours (Dec)'] = diff_df['Hours (Dec)'].round(2)
diff_df['Hours_Difference'] = diff_df['Hours_Difference'].round(2)

# Drop unnecessary columns for cleaner output
diff_df = diff_df.drop(columns=['Date', 'Full Name'])

# Display the rows with significant discrepancies in a table format
print(diff_df.to_string(index=False))

diff_df.to_excel('Short_Report.xlsx')

     Datum    Naam Medewerker  Uren  Hours (Dec)  Hours_Difference
2024-09-02     Roksana Kubala   8.0         4.75              3.25
2024-09-03       Marcin Dudek   8.0         7.97              0.03
2024-09-06 Anisoara Cimpoiesu   8.0         7.40              0.60
2024-09-06       Marcin Dudek   8.0         7.93              0.07
2024-09-06  Orlando Hernandez   8.0         6.13              1.87


C:\Users\boyan.todorov\AppData\Local\Temp\ipykernel_14080\2613005928.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_df['Datum'] = pd.to_datetime(diff_df['Datum']).dt.strftime('%Y-%m-%d')
C:\Users\boyan.todorov\AppData\Local\Temp\ipykernel_14080\2613005928.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_df['Hours (Dec)'] = diff_df['Hours (Dec)'].round(2)
C:\Users\boyan.todorov\AppData\Local\Temp\ipykernel_14080\2613005928.py:21: SettingWithCopyWarning: 
A value is trying to be set on a c